In [3]:
library(MultiBD)
library(MCMCpack)
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install(c("graph", "Rgraphviz"))
# biocLite("graph")
# biocLite("Rgraphviz")

Loading required package: coda

Loading required package: MASS

##
## Markov Chain Monte Carlo Package (MCMCpack)

## Copyright (C) 2003-2020 Andrew D. Martin, Kevin M. Quinn, and Jong Hee Park

##
## Support provided by the U.S. National Science Foundation

## (Grants SES-0350646 and SES-0350613)
##



## Functions

In [ ]:
loglik_sir <- function(param, data) {
    alpha <- exp(param[1]) # Rates must be non-negative 
    beta <- exp(param[2])

    # Set-up SIR model
    drates1 <- function(a, b) { 0 }
    brates2 <- function(a, b) { 0 }
    drates2 <- function(a, b) { alpha * b     }
    trans12 <- function(a, b) { beta  * a * b }

    sum(sapply(1:(nrow(data) - 1), # Sum across all time steps k 
        function(k) {
            log(
                dbd_prob( # Compute the transition probability matrix
                    #t = data$Days[k + 1] - data$Days[k], # Time increment
                    t = 1,
                    a0 = data$S[k], b0 = data$I[k], # From: S(t_k), I(t_k) 
                    drates1, brates2, drates2, trans12,
                    a = data$S[k + 1], B = data$S[k] + data$I[k] - data$S[k + 1], 
                    computeMode = 4, nblocks = 80 # Compute using 4 threads
                )[1, data$I[k + 1] + 1] # To: S(t_(k+1)), I(t_(k+1)) 
            )
        }))
}

In [33]:
logprior <- function(param) {
    log_alpha <- param[1]
    log_beta <- param[2]
    
    dnorm(log_alpha, mean = 0, sd = 100, log = TRUE) + dnorm(log_beta, mean = 0, sd = 100, log = TRUE)
}

In [34]:
loss <- function(alpha, beta, dataset, last_case) {
    loss = 0
    l = len(dataset)
    last = last_case
    for (day in 1:l) {
        S = dataset$S[day]
        I = dataset$I[day]
        D = dataset$D[day]
        n = S + I + D
        pred = last_case + beta * S * I / n - gamma * I
        temp_loss = (log((dataset[day] + 1)) - log(pred + 1))^2
        loss = loss + temp_loss
        last_case = pred
    }
    loss = loss/l
}

In [35]:
setwd("/home/ubuntu/covid19-severity-prediction/county_csv")
csv_names = list.files(pattern = "*")
csv_nums = length(csv_names)

In [ ]:
for (iter in 1:10) {
    total_valid = 0
    total_test = 0
    for (csv in csv_names) {
        s = paste0("/home/ubuntu/covid19-severity-prediction/county_csv/", csv)
        dir = unlist(strsplit(s, split='.', fixed=TRUE))[1]
        data = read.csv(dir, head = TRUE) 
        train_data = data[1:98,]
        valid_data = data[99:114,]
        test_data = data[115:129,]
        alpha0 <- 3.39
        beta0  <- 0.0212
        post_sample <- MCMCmetrop1R(fun = function(param) { loglik_sir(param, data) + logprior(param)},
                        theta.init = log(c(alpha0, beta0)),
                           mcmc = iter, burnin = 20)
        alpha <- mean(exp(post_sample[,1]))
        beta <- mean(exp(post_sample[,2]))
        last_case = data[98,]
        valid_loss = loss(alpha, beta, valid_data, last_case)
        test_loss = loss(alpha, beta, test_data, last_case)
    }
    total_valid = total_valid / csv_nums
    total_test = total_test / csv_nums
    print(paste0("MCMC num = ", toString(iter), ";  valid error = ", toString(total_valid), ";  valid error = ", toString(total_test)))
}

In [2]:
s = paste0("/home/ubuntu/covid19-severity-prediction/county_csv/", csv_names[1])
dir = unlist(strsplit(s, split='.', fixed=TRUE))[1]
data = read.csv(dir, head = TRUE) 
train_data = data[1:98,]
valid_data = data[99:114,]
test_data = data[115:129,]
last_data = data[98,]
print(test_data$Days[1])

ERROR: Error in paste0("/home/ubuntu/covid19-severity-prediction/county_csv/", : object 'csv_names' not found


In [37]:
alpha0 <- 3.39
beta0  <- 0.0212

In [ ]:
post_sample <- MCMCmetrop1R(fun = function(param) {loglik_sir(param, data) + logprior(param)},
                        theta.init = log(c(alpha0, beta0)),
                           mcmc = 20, burnin = 10)

In [41]:
alpha <- mean(exp(post_sample[,1]))
beta <- mean(exp(post_sample[,2]))

In [42]:
alpha
beta

[1] 1.731247

[1] 0.01035141

In [ ]:
loss()